1)Answer:-
Minimize Σ(runs i 1 to n)Σ(runs j 1 to n)Σ(runs k 1 to n)Σ(runs l 1 to n)C(kl)*d(ij)*x(ik)*x(jl) all i,j,k,l={1,2,...n} and x(ik)*x(jl) = y(ijkl); ( where c is cost k th source to l th location and d is demand from i th location to jth loication,x(ij)show the factory i located at locvation j) 
     Σ(i runs 1 to n)y(ijkl) = x(kl) where j,k,l = {1,2,...n}
     Σ(j runs 1 to n)y(ijkl) = x(kl) where i,k,l = {1,2,...n}
     y(ijkl) = y(klij)    where  i,j,k,l = {1,2,...n}
     y(ijkl) >= 0,   i,j,k,l = {1,2,...n}
     x = (x(ij))nxn 

2)solution:
constraints :- n^4 + 2n^3+ 2n
 Binary variable:- n^2; Continuous Variable:-n^4

In [1]:
!pip install -q pyomo

In [2]:
from pyomo.environ import * 

In [3]:
import numpy as np


In [4]:
import pandas as pd

In [5]:
dataq = np.loadtxt('lab7_ex1_q.txt', delimiter=',')
print(dataq)

[[ 0. 18. 22. 19. 23. 20. 18. 19.]
 [21.  0. 20.  0. 19.  0. 22. 20.]
 [22. 23.  0.  0.  0. 17. 16. 24.]
 [17. 25.  0.  0. 21. 22. 20. 17.]
 [12. 19. 18. 19.  0. 21. 21. 23.]
 [20.  0.  0. 17. 21.  0. 20.  0.]
 [22. 24. 28. 16. 23.  0.  0. 19.]
 [23. 29. 20. 17. 24. 24. 23.  0.]]


In [6]:
datac = np.loadtxt('lab7_ex1_c.txt', delimiter=',')
print(datac)


[[ 0. 11. 13.  9. 19. 20. 27. 13.]
 [11.  0.  8.  9. 11. 19. 10. 15.]
 [13.  8.  0. 18. 19. 19. 27. 27.]
 [ 9.  9. 18.  0. 19. 20. 10. 20.]
 [19. 11. 25. 19.  0. 21. 17. 26.]
 [20. 13. 20. 20. 21.  0. 28. 14.]
 [27. 10. 17. 10. 17. 28.  0. 12.]
 [13. 15. 27. 20. 19. 14. 12.  0.]]


In [7]:
model_lab7_ex1 = ConcreteModel()

In [8]:
M = (dataq.shape[0])
print(M)

8


In [9]:
N = (dataq.shape[1])
print(N)

8


In [10]:
row_indices= range(M)
print(row_indices)
col_indices = range(N)


range(0, 8)


In [11]:
model_lab7_ex1.x = Var(row_indices,col_indices,domain= NonNegativeIntegers)
model_lab7_ex1.y = Var(range(8),range(8),range(8),range(8),domain = NonNegativeIntegers)

In [12]:
model_lab7_ex1.constraints =  ConstraintList()


In [13]:
for i in row_indices:
  model_lab7_ex1.constraints.add(expr = sum(model_lab7_ex1.x[i,j] for j in col_indices) == 1)

In [14]:
for j in col_indices:
  model_lab7_ex1.constraints.add(expr = sum(model_lab7_ex1.x[i,j] for i in row_indices) == 1)

In [15]:
for j in range(8):
  for k in range(8):
    for l in range(8):
      model_lab7_ex1.constraints.add(expr = sum(model_lab7_ex1.y[i,j,k,l] for i in range(8)) == model_lab7_ex1.x[k,l])
for i in range(8):
  for k in range(8):
    for l in range(8):
      model_lab7_ex1.constraints.add(expr = sum(model_lab7_ex1.y[i,j,k,l] for j in range(8)) == model_lab7_ex1.x[k,l])
for i in range(8):
  for j in range(8):
    for k in range(8):
      for l in range(8):
        model_lab7_ex1.constraints.add(expr = (model_lab7_ex1.y[i,j,k,l]-model_lab7_ex1.y[k,l,i,j]) == 0)
        model_lab7_ex1.constraints.add(expr = model_lab7_ex1.y[i,j,k,l] >= 0)

In [16]:
model_lab7_ex1.objective = Objective(expr=sum(model_lab7_ex1.y[i,j,k,l]*dataq[i][k]*datac[j][l] for i in range(8) for j in range(8) for k in range(8) for l in range(8)),sense = minimize)

In [17]:
model_lab7_ex1.pprint()

Streaming output truncated to the last 5000 lines.
        4235 :   0.0 :                                                                                        y[3,0,7,5] - y[7,5,3,0] :   0.0 :   True
        4236 :   0.0 :                                                                                                     y[3,0,7,5] :  +Inf :   True
        4237 :   0.0 :                                                                                        y[3,0,7,6] - y[7,6,3,0] :   0.0 :   True
        4238 :   0.0 :                                                                                                     y[3,0,7,6] :  +Inf :   True
        4239 :   0.0 :                                                                                        y[3,0,7,7] - y[7,7,3,0] :   0.0 :   True
        4240 :   0.0 :                                                                                                     y[3,0,7,7] :  +Inf :   True
        4241 :   0.0 :                     

In [18]:
!apt-get install -y -qq coinor-cbc

In [19]:
opt_cbc= SolverFactory('cbc',)

In [20]:
result = opt_cbc.solve(model_lab7_ex1)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [21]:
print('\nObjective=',model_lab7_ex1.objective())
print("Dicision Variable:")
for i in range(8):
  for j in range(8):
    if (model_lab7_ex1.x[i,j].value != 0):
      print("facility",i+1,"is opened at location ", j+1)


Objective= 14889.0
Dicision Variable:
facility 1 is opened at location  2
facility 2 is opened at location  8
facility 3 is opened at location  7
facility 4 is opened at location  6
facility 5 is opened at location  1
facility 6 is opened at location  3
facility 7 is opened at location  5
facility 8 is opened at location  4


9)
optimal cost= 14889.0

facility 1 is opened at location  2;
facility 2 is opened at location  8;
facility 3 is opened at location  7;
facility 4 is opened at location  6;
facility 5 is opened at location  1;
facility 6 is opened at location  3;
facility 7 is opened at location  5;
facility 8 is opened at location  4;

10) solution

In [22]:
model_lab7_ex1.x.domain= NonNegativeReals

In [23]:
model_lab7_ex1.y.domain = NonNegativeReals

In [24]:
!apt-get install -y -qq coinor-cbc

In [25]:
opt_cbc= SolverFactory('cbc',)

In [26]:
result = opt_cbc.solve(model_lab7_ex1)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [27]:
print(model_lab7_ex1.objective())

11790.875004574149


In [28]:
print('\nObjective=',model_lab7_ex1.objective())
print("Dicision Variable:")
for i in range(8):
  for j in range(8):
    if (model_lab7_ex1.x[i,j].value != 0):
      print('Value of Decision Variable x[',i+1,'][',j+1,']=',model_lab7_ex1.x[i,j].value)
      


Objective= 11790.875004574149
Dicision Variable:
Value of Decision Variable x[ 1 ][ 1 ]= 0.084983499
Value of Decision Variable x[ 1 ][ 2 ]= 0.33367696
Value of Decision Variable x[ 1 ][ 3 ]= 0.048316653
Value of Decision Variable x[ 1 ][ 4 ]= 0.10039981
Value of Decision Variable x[ 1 ][ 5 ]= 0.051444116
Value of Decision Variable x[ 1 ][ 6 ]= 0.25047975
Value of Decision Variable x[ 1 ][ 7 ]= 0.066562267
Value of Decision Variable x[ 1 ][ 8 ]= 0.064136939
Value of Decision Variable x[ 2 ][ 1 ]= 0.19073548
Value of Decision Variable x[ 2 ][ 2 ]= 0.026261001
Value of Decision Variable x[ 2 ][ 3 ]= 0.11864323
Value of Decision Variable x[ 2 ][ 4 ]= 0.1917243
Value of Decision Variable x[ 2 ][ 5 ]= 0.17399296
Value of Decision Variable x[ 2 ][ 6 ]= 0.022830959
Value of Decision Variable x[ 2 ][ 7 ]= 0.15180168
Value of Decision Variable x[ 2 ][ 8 ]= 0.1240104
Value of Decision Variable x[ 3 ][ 1 ]= 0.10388789
Value of Decision Variable x[ 3 ][ 2 ]= -4.6709609e-15
Value of Decision Varia

Optimal soution :- 11790.875004574149
Value of Decision Variable x[ 1 ][ 1 ]= 0.084983499
Value of Decision Variable x[ 1 ][ 2 ]= 0.33367696
Value of Decision Variable x[ 1 ][ 3 ]= 0.048316653
Value of Decision Variable x[ 1 ][ 4 ]= 0.10039981
Value of Decision Variable x[ 1 ][ 5 ]= 0.051444116
Value of Decision Variable x[ 1 ][ 6 ]= 0.25047975
Value of Decision Variable x[ 1 ][ 7 ]= 0.066562267
Value of Decision Variable x[ 1 ][ 8 ]= 0.064136939
Value of Decision Variable x[ 2 ][ 1 ]= 0.19073548
Value of Decision Variable x[ 2 ][ 2 ]= 0.026261001
Value of Decision Variable x[ 2 ][ 3 ]= 0.11864323
Value of Decision Variable x[ 2 ][ 4 ]= 0.1917243
Value of Decision Variable x[ 2 ][ 5 ]= 0.17399296
Value of Decision Variable x[ 2 ][ 6 ]= 0.022830959
Value of Decision Variable x[ 2 ][ 7 ]= 0.15180168
Value of Decision Variable x[ 2 ][ 8 ]= 0.1240104
Value of Decision Variable x[ 3 ][ 1 ]= 0.10388789
Value of Decision Variable x[ 3 ][ 2 ]= -4.6709609e-15
Value of Decision Variable x[ 3 ][ 3 ]= 0.17341885
Value of Decision Variable x[ 3 ][ 4 ]= 0.20991576
Value of Decision Variable x[ 3 ][ 5 ]= 0.15765049
Value of Decision Variable x[ 3 ][ 6 ]= 0.12777006
Value of Decision Variable x[ 3 ][ 7 ]= 0.12970289
Value of Decision Variable x[ 3 ][ 8 ]= 0.09765406
Value of Decision Variable x[ 4 ][ 1 ]= 0.07739104
Value of Decision Variable x[ 4 ][ 2 ]= 0.0017389244
Value of Decision Variable x[ 4 ][ 3 ]= 0.16883651
Value of Decision Variable x[ 4 ][ 4 ]= 0.041916945
Value of Decision Variable x[ 4 ][ 5 ]= 0.22827876
Value of Decision Variable x[ 4 ][ 6 ]= 0.13528148
Value of Decision Variable x[ 4 ][ 7 ]= 0.11864323
Value of Decision Variable x[ 4 ][ 8 ]= 0.22791312
Value of Decision Variable x[ 5 ][ 1 ]= 0.20537663
Value of Decision Variable x[ 5 ][ 2 ]= 0.14760491
Value of Decision Variable x[ 5 ][ 3 ]= 0.15701317
Value of Decision Variable x[ 5 ][ 4 ]= 0.043650923
Value of Decision Variable x[ 5 ][ 5 ]= 0.038651246
Value of Decision Variable x[ 5 ][ 6 ]= 0.0772711
Value of Decision Variable x[ 5 ][ 7 ]= 0.15673067
Value of Decision Variable x[ 5 ][ 8 ]= 0.17370134
Value of Decision Variable x[ 6 ][ 1 ]= 0.022830959
Value of Decision Variable x[ 6 ][ 3 ]= 0.19583209
Value of Decision Variable x[ 6 ][ 4 ]= 0.019085987
Value of Decision Variable x[ 6 ][ 5 ]= 0.26693
Value of Decision Variable x[ 6 ][ 6 ]= 0.32892201
Value of Decision Variable x[ 6 ][ 7 ]= 0.06176476
Value of Decision Variable x[ 6 ][ 8 ]= 0.1046342
Value of Decision Variable x[ 7 ][ 1 ]= 0.063140624
Value of Decision Variable x[ 7 ][ 2 ]= 0.31854682
Value of Decision Variable x[ 7 ][ 3 ]= 0.06176476
Value of Decision Variable x[ 7 ][ 4 ]= 0.26544384
Value of Decision Variable x[ 7 ][ 5 ]= 0.083052425
Value of Decision Variable x[ 7 ][ 6 ]= 0.023820054
Value of Decision Variable x[ 7 ][ 7 ]= 0.063140624
Value of Decision Variable x[ 7 ][ 8 ]= 0.12109086
Value of Decision Variable x[ 8 ][ 1 ]= 0.25165388
Value of Decision Variable x[ 8 ][ 2 ]= 0.17217138
Value of Decision Variable x[ 8 ][ 3 ]= 0.076174743
Value of Decision Variable x[ 8 ][ 4 ]= 0.12786244
Value of Decision Variable x[ 8 ][ 5 ]= 9.6620689e-13
Value of Decision Variable x[ 8 ][ 6 ]= 0.033624599
Value of Decision Variable x[ 8 ][ 7 ]= 0.25165388
Value of Decision Variable x[ 8 ][ 8 ]= 0.086859086


11)Answer:-
Optimal cost of integer and real value are not same. becaused in real value it can take non integer extreme feasible point.so its show perfact optimal solution.in integer domain it have to find integer extreme point,so it have maximum optimal cost.